# Language dataset utils

In [ ]:
#|default_exp lang.dataset

In [ ]:
#|hide
from fastcore.test import *
from nbdev.showdoc import *

In [ ]:
#|export


def alpaca2sharegpt(example):
    messages = []
    if "instruction" in example:
        messages.append({"from": "system", "value": example["instruction"]})
    messages.append({"from": "human", "value": example["input"]})
    messages.append({"from": "gpt", "value": example["output"]})
    return {"messages": messages}


def alpaca2openai(example):
    messages = []
    if "instruction" in example:
        messages.append({"role": "system", "content": example["instruction"]})
    messages.append({"role": "user", "content": example["input"]})
    messages.append({"role": "assistant", "content": example["output"]})
    return {"messages": messages}


def sharegpt2openai_message(message):
    role = {"system": "system", "human": "user", "gpt": "assistant"}[message["from"]]
    return {"role": role, "content": message["value"]}


def openai2sharegpt_message(message):
    frm = {"system": "system", "user": "human", "assistant": "gpt"}[message["role"]]
    return {"from": frm, "value": message["content"]}


def sharegpt2openai(example):
    return {
        "messages": [sharegpt2openai_message(message) for message in example["messages"]],
    }


def openai2sharegpt(example):
    return {
        "messages": [openai2sharegpt_message(message) for message in example["messages"]],
    }

In [ ]:
alpaca_example = {
    "instruction": "You are a helpful assistant.",
    "input": "What is love?",
    "output": "Baby, don't hurt me!"
}

In [ ]:
sharegpt_example = alpaca2sharegpt(alpaca_example)
sharegpt_example

{'messages': [{'from': 'system', 'value': 'You are a helpful assistant.'},
  {'from': 'human', 'value': 'What is love?'},
  {'from': 'gpt', 'value': "Baby, don't hurt me!"}]}

In [ ]:
openai_example = alpaca2openai(alpaca_example)
openai_example

{'messages': [{'role': 'system', 'content': 'You are a helpful assistant.'},
  {'role': 'user', 'content': 'What is love?'},
  {'role': 'assistant', 'content': "Baby, don't hurt me!"}]}

In [ ]:
test_eq(openai_example, sharegpt2openai(sharegpt_example))

In [ ]:
test_eq(sharegpt_example, openai2sharegpt(openai_example))

In [ ]:
#|export

def partition_input_output_messages(example: dict) -> dict:
    result = {}
    last_message = example["messages"][-1]
    if last_message['role'] == 'assistant':
        result["input"] = example["messages"][:-1]
        result["output"] = example["messages"][-1:]
    else:
        result["input"] = example["messages"]
        result["output"] = []
    return result
    

In [ ]:
#|hide
example_with_completion = {
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What is love?"},
        {'role': 'assistant', 'content': "Baby, don't hurt me!"},
    ]
}

result = partition_input_output_messages(example_with_completion)
test_eq(len(result["input"]), len(example_with_completion["messages"]) - 1)
test_eq(len(result["output"]), 1)


example_with_no_completion = {
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What is love?"},
    ]
}
result = partition_input_output_messages(example_with_no_completion)
test_eq(len(result["input"]), len(example_with_no_completion["messages"]))
test_eq(len(result["output"]), 0)

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()